In [3]:
import numpy as np
from qiskit import QuantumCircuit, transpile
from qiskit.quantum_info import Kraus, DensityMatrix, partial_trace
from qiskit_aer import AerSimulator
from qiskit_aer.noise import NoiseModel, QuantumError
import matplotlib.pyplot as plt
import os



def format_p_prime_for_display(p):

    p_str = f"{p:g}"
    if 'e' in p_str:
        base, exponent = p_str.split('e')
        exponent = int(exponent)
        return f"{base} \\times 10^{{{exponent}}}" 
    else:
        return p_str
def format_p_prime_for_filename(p):

    return f"{p:g}".replace('.', 'p').replace('e-', 'em')


def conditional_x_noise(p):
    I = np.eye(2)
    X = np.array([[0, 1], [1, 0]])
    P0 = np.array([[1, 0], [0, 0]])
    P1 = np.array([[0, 0], [0, 1]])
    
    K1 = np.sqrt(1 - p) * np.kron(I, I)
    K2 = np.sqrt(p) * (np.kron(I, P0) + np.kron(X, P1))
    return Kraus([K1, K2])



def sx_noise(p):
    I = np.eye(2, dtype=np.complex128)
    X = np.array([[0, 1], [1, 0]], dtype=np.complex128)
    K1 = np.sqrt(1 - p) * I
    K2 = np.sqrt(p) * X
    return Kraus([K1, K2])



def rz_noise(p):
    I = np.eye(2, dtype=np.complex128)
    Z = np.array([[1, 0], [0, -1]], dtype=np.complex128)
    K1 = np.sqrt(1 - p) * I
    K2 = np.sqrt(p) * Z
    return Kraus([K1, K2])



GLOBAL_BASIS_GATES = ["cx", "sx", "rz"]

def custom_transpile(circuit, backend=None, optimization_level=1):
    return transpile(
        circuit,
        basis_gates=GLOBAL_BASIS_GATES,
        backend=backend,
        optimization_level=optimization_level
    )

# ===================================================================
def apply_global_depolarizing(rho, p):
    n = rho.num_qubits
    dim = 2 ** n
    identity = np.eye(dim, dtype=np.float64) / dim
    return DensityMatrix(rho.data + p * (identity - rho.data))

p_init = 0.9
rho_single = DensityMatrix(np.diag([p_init, 1 - p_init]))

def build_U(n):
    qc = QuantumCircuit(n)
    if n == 1:
        return qc
    
    qc.x(n-1)
    for i in range(n-2, -1, -1):
        controls = list(range(i+1, n))
        qc.mcx(controls, i)
    
    if n > 1:
        qc.mcx(list(range(n-1)), n-1)
    
    qc.x(n-1)
    for i in range(n-1):
        controls = list(range(i+1, n))
        qc.mcx(controls, i)
    
    qc.x(n-1)

    decomposed_qc = transpile(qc, basis_gates=GLOBAL_BASIS_GATES, optimization_level=0)
    return decomposed_qc

FITTED_Q_VALUES = { 3: 0.496, 4: 0.45, 5: 0.54 }

def simulate_with_tracking(n, noise_model=None, num_rounds=20, 
                         use_custom_noise=False, p_prime=0.0,
                         use_fitted_model=False):
    populations = []
    current_rho = rho_single
    for _ in range(n-1):
        current_rho = current_rho.tensor(rho_single)
    
    populations.append(partial_trace(current_rho, list(range(1, n))).data[0,0])
    
    decomposed_U = build_U(n)
    
    if use_custom_noise or use_fitted_model:
        ops = decomposed_U.count_ops()
        L = sum(ops.values())
        cnot_count = ops.get('cx', 0)
        d = 2**n
        if use_fitted_model:
            q = FITTED_Q_VALUES[n]
            p = p_prime * cnot_count * (1 - q)
        else:
            q = ( (d**2) / 4 - 1 ) / ( d**2 - 1 )
            p = p_prime * (cnot_count - cnot_count * q)
        
        p = max(0.0, min(p, 1.0))
    else:
        p = 0.0
    
    for _ in range(num_rounds):
        qc = QuantumCircuit(n)
        qc.set_density_matrix(current_rho)
        qc.compose(decomposed_U, inplace=True)
        qc.save_state()
        
        simulator = AerSimulator(method='density_matrix', noise_model=None if (use_custom_noise or use_fitted_model) else noise_model)
        tqc = transpile(qc, simulator)
        result = simulator.run(tqc).result()
        new_rho = DensityMatrix(result.data(0)['density_matrix'])
        
        if (use_custom_noise or use_fitted_model) and p > 0:
            new_rho = apply_global_depolarizing(new_rho, p)
        
        pop = partial_trace(new_rho, list(range(1, n))).data[0,0]
        populations.append(pop)
        traced = partial_trace(new_rho, [n-1])
        current_rho = rho_single.tensor(traced)
    
    return populations


def run_and_plot_simulation(n, p_prime, num_rounds=200):

    columnwidth_pt = 246.0
    pt_to_inch = 1.0 / 72.27
    fig_width_inch = columnwidth_pt * pt_to_inch
    golden_ratio = (np.sqrt(5) - 1.0) / 2.0
    fig_height_inch = fig_width_inch * golden_ratio
    
    font_size_pt = 8.0
    plt.rcParams.update({
        "text.usetex": False,
        
        "font.family": "serif",
        "font.serif": ["Times New Roman"],
        "mathtext.fontset": "stix", 
        
        "font.size": font_size_pt,
        "axes.labelsize": font_size_pt,
        "xtick.labelsize": font_size_pt,
        "ytick.labelsize": font_size_pt,
        "legend.fontsize": font_size_pt,
        "axes.linewidth": 0.5,
        "lines.linewidth": 1.2,
        "xtick.major.width": 0.5,
        "ytick.major.width": 0.5,
    })

    p_cx = p_prime
    p_sx = p_rz = 0

    noise_model = NoiseModel()
    noise_model.add_all_qubit_quantum_error(QuantumError(sx_noise(p_sx)), 'sx')
    noise_model.add_all_qubit_quantum_error(QuantumError(rz_noise(p_rz)), 'rz')
    noise_model.add_all_qubit_quantum_error(QuantumError(conditional_x_noise(p_cx)), 'cx')

    print(f"Running simulation for n={n} with p={p_prime:g}...")
    ideal = simulate_with_tracking(n, num_rounds=num_rounds)
    noisy = simulate_with_tracking(n, noise_model=noise_model, num_rounds=num_rounds)
    global_dep = simulate_with_tracking(n, use_custom_noise=True, p_prime=p_prime, num_rounds=num_rounds)
    print("Simulation complete.")

    fig, ax = plt.subplots(figsize=(fig_width_inch, fig_height_inch))

    ax.plot(ideal, color='blue', linestyle='-', label='Ideal')
    ax.plot(noisy, color='red', linestyle='--', label='Physical Noise')
    ax.plot(global_dep, color='green', linestyle=':', label='Theoretical Global Dep.')
    
    p_prime_display = format_p_prime_for_display(p_prime)
    ax.set_title(f"$n={n}$, $p={p_prime_display}$")
    
    ax.set_xlabel('Cooling Rounds')
    ax.set_ylabel('Ground State Population')
    ax.legend(frameon=False)
    ax.grid(True, linestyle='--', alpha=0.6)
    ax.set_ylim(top=1)
    fig.tight_layout(pad=0.1)

    output_dir = "D:/quantum_sim_figures"
    os.makedirs(output_dir, exist_ok=True)
    

    p_prime_filename = format_p_prime_for_filename(p_prime)
    file_name = f"simulation_n{n}_p_prime_{p_prime_filename}_no_latex.pdf"
    
    output_path = os.path.join(output_dir, file_name)
    
    plt.savefig(output_path, format='pdf', bbox_inches='tight')
    print(f"Figure saved to: {output_path}")
    plt.close(fig)

if __name__ == "__main__":
    
    #run_and_plot_simulation(n=5, p_prime=0.00002, num_rounds=250)#0.9
    #run_and_plot_simulation(n=8, p_prime=0.0000005, num_rounds=250)#0.9
    #run_and_plot_simulation(n=3, p_prime=0.0003, num_rounds=250) #0.7
    #run_and_plot_simulation(n=4, p_prime=0.00015, num_rounds=250) #0.7
    #run_and_plot_simulation(n=5, p_prime=0.00001, num_rounds=250)
    #run_and_plot_simulation(n=6, p_prime=0.00001, num_rounds=250)
    run_and_plot_simulation(n=7, p_prime=0.000005, num_rounds=250)

Running simulation for n=7 with p=5e-06...
Simulation complete.
Figure saved to: D:/quantum_sim_figures\simulation_n7_p_prime_5em06_no_latex.pdf


C:\ProgramData\anaconda3\Lib\site-packages\matplotlib\cbook.py:1762: ComplexWarning: Casting complex values to real discards the imaginary part
  return math.isfinite(val)
C:\ProgramData\anaconda3\Lib\site-packages\matplotlib\cbook.py:1398: ComplexWarning: Casting complex values to real discards the imaginary part
  return np.asarray(x, float)
